In [8]:
import pandas as pd
import  random
import schedule
import time
from math import floor
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
from datetime import datetime, date, timedelta

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodadosorigem =  "MERCADO_STAGE" 
bancodadosdestino = "MERCADO_DATAWAREHOUSE"
driver = "ODBC Driver 17 for SQL Server"

conexao_origem  = f"mssql+pyodbc://{server}/{bancodadosorigem}?driver={driver}&Trusted_Connection=yes"
conexao_destino = f"mssql+pyodbc://{server}/{bancodadosdestino}?driver={driver}&Trusted_Connection=yes"
engine_origem  = create_engine(conexao_origem) # pool_size=10, max_overflow=30
engine_destino = create_engine(conexao_destino)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print(f"Query teste executada com sucesso, conexão com o banco de dados {engine} bem-sucedida")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
testar_conexao_sqlalchemy(engine_origem) 
testar_conexao_sqlalchemy(engine_destino)

tipo_dados_st_categoria = {
                'IDCATEGORIA':    INTEGER,
                'NOME':           VARCHAR(50),
                'DESCRICAO':      VARCHAR(150),
                'DATA_CRIACAO':   DATE,
                'DATA_ATLZ':      DATE,
                'VERSAO':         INTEGER,
}

tipo_dados_st_cfop = {
                'IDCODCFOP':      INTEGER,
                'CODCFOP':        VARCHAR(20),
                'DESCRICAO':      VARCHAR(100),
}

tipo_dados_st_cidade = {
                'IDCIDADE':      INTEGER,
                'NOME':          VARCHAR(50),
                'ID_ESTADO':     INTEGER,
}

tipo_dados_st_cliente = {
                'IDCLIENTE':     INTEGER,
                'NOME':          VARCHAR(100),
                'CPF':           VARCHAR(11),
                'EMAIL':         VARCHAR(50),
                'TELEFONE':      VARCHAR(20),
                'DATA_NASC':     DATE,
                'GENERO':        CHAR(1),
                'ENDERECO':      VARCHAR(100),
                'ID_CIDADE':     INTEGER,
                "DATA_CRIACAO":  DATE,
                "DATA_ATLZ":     DATE,
                "VERSAO":        INTEGER,
}

tipo_dados_st_departamento = {
                'ID_DEPARTAMENTO':  INTEGER,
                'NOME':             VARCHAR(100),   

}

tipo_dados_st_estado = {
                'IDESTADO':   INTEGER,
                'ESTADO':     VARCHAR(25),
                'UF':         VARCHAR(2),
                'REGIAO':     VARCHAR(20),
                'PAIS':       VARCHAR(20),

}

tipo_dados_st_forma_pagamento = {
                'IDFORMAPGMT':   INTEGER,
                'FORMA':         VARCHAR(50),
                'DATA_CRIACAO':  DATE,
                'DATA_ATLZ':     DATE,
                'VERSAO':        INTEGER,

}

tipo_dados_st_fornecedores = {
                'IDFORNECEDOR': INTEGER,
                "NOME":         VARCHAR(100),
                "CNPJ":         VARCHAR(14),
                "TELEFONE":     VARCHAR(20),
                "EMAIL":        VARCHAR(100),
                "ENDERECO":     VARCHAR(100),
                "ID_CIDADE":    INTEGER,
                "DATA_CRIACAO": DATE,
                "DATA_ATLZ":    DATE,
                "VERSAO":       INTEGER,
}

tipo_dados_st_funcionarios = {
                'IDFUNCIONARIO':     INTEGER,
                'NOME':              VARCHAR(100),
                'CPF':               VARCHAR(11),
                'CARGO':             VARCHAR(50),
                'SALARIO':           NUMERIC(18,2),
                'ENDERECO':          VARCHAR(100),
                'ID_CIDADE':         INTEGER,
                'ID_LOJA':           INTEGER,
                'ID_GERENTE':        INTEGER,
                'ID_DEPARTAAMENTO':  INTEGER,
                'SITUACAO':          VARCHAR(10),
                'DATA_ADMISSAO':     DATE,
                'DATA_DEMISSAO':     DATE,
                'DATA_CRIACAO':      DATE,
                'DATA_ATLZ':         DATE,
                'VERSAO':            INTEGER,
}

tipo_dados_st_gerente = {
                'IDGERENTE':         INTEGER,
                'NOME':              VARCHAR(100),
                'ID_LOJA':           INTEGER,
                'ID_DEPARTAAMENTO':  INTEGER,
                'DATA_CRIACAO':      DATE,
                'DATA_ATLZ':         DATE,
                'VERSAO':            INTEGER,
}

tipo_dados_st_item_prod_fornecedor = {
                        'IDPRODFORNECEDOR': INTEGER,
                        'ID_PRODUTO':       INTEGER,
                        'ID_FORNECEDOR':    INTEGER,

}


tipo_dados_st_itemnota = {
                        'IDITEMNOTA':     INTEGER,
                        'ID_NOTAFISCAL':  INTEGER,
                        'ID_PRODUTO':     INTEGER,
                        'QUANTIDADE':     INTEGER,
                        'PRECO_UNIT':     NUMERIC(18,2),
                        'TOTAL_ITEM':     NUMERIC(18,2),
                        'DATA_CRIACAO':   DATE,
                        'DATA_ATLZ':      DATE,
                        'VERSAO':         INTEGER,
}

tipo_dados_st_loja = {
                'IDLOJA':        INTEGER,
                'NOME_LOJA':     VARCHAR(100),
                'ENDERECO':      VARCHAR(100),
                'ID_CIDADE':     INTEGER,
                'DATA_CRIACAO':  DATE,
                'DATA_ATLZ':     DATE,
                'VERSAO':        INTEGER,
}

tipo_dados_st_notafiscal = {
                'IDNOTAFISCAL':   INTEGER,
                'NUMERONF':       INTEGER,
                'DATA':           DATE,
                'TOTAL_NOTA':     NUMERIC(18,2),
                'ID_FORMA_PGMT':  INTEGER,
                'ID_CLIENTE':     INTEGER,
                'ID_LOJA':        INTEGER,
                'ID_CFOP':        INTEGER,
                'CHAVE_ACESSO':   VARCHAR(100),
                'DATA_CRIACAO':   DATE,
                'DATA_ATLZ':      DATE,
                'VERSAO':         INTEGER,
}

tipo_dados_st_produto = {
                'IDPRODUTO':     INTEGER,
                'NOME':          VARCHAR(100),
                'ID_CATEGORIA':  INTEGER,
                'PRECO_CUSTO':   NUMERIC(18,2),
                'PRECO_UNIT':    NUMERIC(18,2),
                'ID_UNID_MED':   INTEGER,
                "DATA_CRIACAO":  DATE,
                "DATA_ATLZ":     DATE,
                "VERSAO":        INTEGER,
}

tipo_dados_st_venda = {
                'IDVENDA':        INTEGER,
                'DATA':           DATE,
                'ID_NOTAFISCAL':  INTEGER,
                'ID_CLIENTE':     INTEGER,
                'ID_LOJA':        INTEGER,
                'ID_FORMA_PGMT':  INTEGER,
                'TOTAL_VENDA':    NUMERIC(18,2),
                'DATA_CRIACAO':   DATE,
                'DATA_ATLZ':      DATE,
                'VERSAO':         INTEGER,
}

lista_categoria = ['CATEGORIA',
                   ]

                   #'CFOP',
                   #'CIDADE',
                   #'CLIENTE',
                   #'DEPARTAMENTO',
                   #'ESTADO',
                   #'FORMA_PAGAMENTO',
                   #'FORNECEDOR',
                   #'FUNCIONARIO',
                   #'GERENTE',
                   #'ITEM_PROD_FORNECEDOR',
                   #'ITEMNOTA',
                   #'LOJA',
                   #'NOTAFISCAL',
                   #'PRODUTO',
                   #'VENDA']

lista_tipo_dados = [tipo_dados_st_categoria,
                    tipo_dados_st_cfop,
                    tipo_dados_st_cidade,
                    tipo_dados_st_cliente,
                    tipo_dados_st_departamento,
                    tipo_dados_st_estado,
                    tipo_dados_st_forma_pagamento,
                    tipo_dados_st_fornecedores,
                    tipo_dados_st_funcionarios,
                    tipo_dados_st_gerente,
                    tipo_dados_st_item_prod_fornecedor,
                    tipo_dados_st_itemnota,
                    tipo_dados_st_loja,
                    tipo_dados_st_notafiscal,
                    tipo_dados_st_produto,
                    tipo_dados_st_venda]

#del lista_categoria[3]
#del lista_categoria[7]
#del lista_categoria[8]
#del lista_categoria[8]
#del lista_categoria[9]
#del lista_categoria[9]
#display(lista_categoria)
#del lista_tipo_dados[3]
#del lista_tipo_dados[7]
#del lista_tipo_dados[8]
#del lista_tipo_dados[8]
#del lista_tipo_dados[9]
#del lista_tipo_dados[9]

def carga_dw(lista_categoria,conn_fonte,conn_destino,lista_tipo_dados):
     for item in lista_categoria:
        try:
            df = pd.read_sql(f"SELECT * FROM ST_{item}", conn_fonte)
            indice = lista_categoria.index(item)
            tipo_dados = lista_tipo_dados[indice]
            with conn_destino.connect() as conn:
                 conn.execute(text(f"TRUNCATE TABLE DIM_{item}"))
                 conn.commit()
            df.to_sql(name=f'DIM_{item}',con=conn_destino,if_exists='append',index=False,dtype=tipo_dados)
            print(f'Carga tabela {item} feita com sucesso no DW.')
        except Exception as e:
             print(f'Erro ao fazer a carga de {item}. Erro: {e}')

#carga_dw(lista_categoria,engine_origem,engine_destino,lista_tipo_dados)

tipo_dados_FATO_VENDA = {
                'NUMERONF':         INTEGER,
                'ID_CLIENTE':       INTEGER,
                'ID_LOJA':          INTEGER,
                'ID_FORMA_PGMT':    INTEGER,
                'ID_PRODUTO':       INTEGER,
                'DATA':             DATE,
                'QUANTIDADE':       INTEGER,
                'TOTAL_ITEM':       NUMERIC(18,2),
                'TOTAL_CUSTO':      NUMERIC(18,2),
                'LUCRO_TOTAL_ITEM': NUMERIC(18,2),
                
}

def carga_fato_venda(conn_fonte,conn_destino,tipo_dados):
    try:
        df = pd.read_sql(f"SELECT * FROM VW_TABELA_FATO_VENDA", conn_fonte)
        
        with conn_destino.connect() as conn:
             conn.execute(text(f"TRUNCATE TABLE FATO_VENDA"))
             conn.commit()
        df.to_sql(name=f'FATO_VENDA',con=conn_destino,if_exists='append',index=False,dtype=tipo_dados)
        print(f'Carga tabela FATO_VENDA feita com sucesso no DW.')
    except Exception as e:
        print(f'Erro ao fazer a carga de FATO_VENDA. Erro: {e}')

carga_fato_venda(engine_origem,engine_destino,tipo_dados_FATO_VENDA)





Query teste executada com sucesso, conexão com o banco de dados Engine(mssql+pyodbc://DESKTOP-HV85ADA\PROJETO_MERCADO/MERCADO_STAGE?Trusted_Connection=yes&driver=ODBC+Driver+17+for+SQL+Server) bem-sucedida
Query teste executada com sucesso, conexão com o banco de dados Engine(mssql+pyodbc://DESKTOP-HV85ADA\PROJETO_MERCADO/MERCADO_DATAWAREHOUSE?Trusted_Connection=yes&driver=ODBC+Driver+17+for+SQL+Server) bem-sucedida
Carga tabela FATO_VENDA feita com sucesso no DW.
